# Problem 201: Subsets with a Unique Sum

For any set $A$ of numbers, let $\operatorname{sum}(A)$ be the sum of the elements of $A$.
Consider the set $B = \{1,3,6,8,10,11\}$.
There are $20$ subsets of $B$ containing three elements, and their sums are:

\begin{align}
\operatorname{sum}(\{1,3,6\}) &= 10, \nonumber \\
\operatorname{sum}(\{1,3,8\}) &= 12, \nonumber \\
\operatorname{sum}(\{1,3,10\}) &= 14, \nonumber \\
\operatorname{sum}(\{1,3,11\}) &= 15, \nonumber \\
\operatorname{sum}(\{1,6,8\}) &= 15, \nonumber \\
\operatorname{sum}(\{1,6,10\}) &= 17, \nonumber \\
\operatorname{sum}(\{1,6,11\}) &= 18, \nonumber \\
\operatorname{sum}(\{1,8,10\}) &= 19, \nonumber \\
\operatorname{sum}(\{1,8,11\}) &= 20, \nonumber \\
\operatorname{sum}(\{1,10,11\}) &= 22, \nonumber \\
\operatorname{sum}(\{3,6,8\}) &= 17, \nonumber \\
\operatorname{sum}(\{3,6,10\}) &= 19, \nonumber \\
\operatorname{sum}(\{3,6,11\}) &= 20, \nonumber \\
\operatorname{sum}(\{3,8,10\}) &= 21, \nonumber \\
\operatorname{sum}(\{3,8,11\}) &= 22, \nonumber \\
\operatorname{sum}(\{3,10,11\}) &= 24, \nonumber \\
\operatorname{sum}(\{6,8,10\}) &= 24, \nonumber \\
\operatorname{sum}(\{6,8,11\}) &= 25, \nonumber \\
\operatorname{sum}(\{6,10,11\}) &= 27, \nonumber \\
\operatorname{sum}(\{8,10,11\}) &= 29. \nonumber 
\end{align}

Some of these sums occur more than once, others are unique.\
For a set $A$, let $U(A,k)$ be the set of unique sums of $k$-element subsets of $A$, in our example we find $U(B,3) = \{10,12,14,18,21,25,27,29\}$ and $\operatorname{sum}(U(B,3)) = 156$.

Now consider the $100$-element set $S = \{1^2, 2^2, \dots, 100^2\}$.\
S has $100891344545564193334812497256$ $50$-element subsets.

Determine the sum of all integers which are the sum of exactly one of the $50$-element subsets of $S$, i.e. find $\operatorname{sum}(U(S,50))$.

In [1]:
# brute force

from collections import defaultdict
from itertools import combinations
from typing import List, Set, DefaultDict


num_elements = 20
num_subset = 10

S = set([s**2 for s in range(1, num_elements + 1)])


subset_sums: DefaultDict[int, List[Set[int]]] = defaultdict(list)

for subset in combinations(S, num_subset):
    subset_sums[sum(subset)].append(set(subset))

U = [_sum for (_sum, subsets) in subset_sums.items() if len(subsets) == 1]
print(sum(U))

152110


In [23]:
import functools
from itertools import accumulate

num_elements = 100
num_subset = 50

S: List[int] = [s**2 for s in range(1, num_elements + 1)][::-1]
cumulative_S: List[int] = list(accumulate(S, initial=0))


# Returns 0 if sum is not possible to make, 1 if the sume is unique and >1 if the sum is not unique
@functools.cache
def is_unique_sum(
    target_sum: int, set_index: int = 0, num_elements_left: int = num_subset
) -> int:

    max_attainable_sum = (
        cumulative_S[set_index + num_elements_left] - cumulative_S[set_index]
    )
    min_attainable_sum = (
        cumulative_S[-num_elements_left] - cumulative_S[-(num_elements_left + 1)]
        if num_elements_left > 0
        else 0
    )

    match target_sum:
        case 0 if num_elements_left == 0:
            return 1
        case _ if target_sum < min_attainable_sum or target_sum > max_attainable_sum:
            return 0
        case _:
            num_ways_to_make_sum = 0
            for i in range(set_index, num_elements - num_elements_left + 1):
                num_ways_to_make_sum += is_unique_sum(
                    target_sum - S[i], i + 1, num_elements_left - 1
                )
                if num_ways_to_make_sum > 1:
                    return num_ways_to_make_sum
            return num_ways_to_make_sum


is_unique_sum.cache_clear()
_max_sum = sum(S[:num_subset])
_min_sum = sum(S[-num_subset:])
U = [n for n in range(_min_sum, _max_sum + 1) if is_unique_sum(n) == 1]
print(sum(U))

115039000
